# Gradient Asymmetry Stopping Experiment

**Complete Experiment Suite - All 16 Learning Rate Combinations**

**Learning Rate Combinations:**
- Actor LRs: `[0.0001, 0.001, 0.01, 0.1]`
- Critic LRs: `[0.0001, 0.001, 0.01, 0.1]`
- Total: **16 experiments**

---

## Features:
- Auto-saves to Google Drive every 100 episodes
- Auto-restores from Drive on session restart
- Crash recovery built-in
- Early stopping when all actors stop updating

## Cell 1: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted!")

## Cell 2: Upload and Extract Experiment Files

**Option A:** Upload `colab_experiment.zip` when prompted

**Option B:** If you already uploaded to Drive, skip the upload prompt

In [ ]:
import os
import zipfile

# Check if files already exist
if os.path.exists('/content/stopping_experiment_run/mec_env.py'):
    print("Experiment files already present!")
else:
    # Try to find zip in Drive first
    drive_zip = '/content/drive/MyDrive/colab_experiment.zip'
    
    if os.path.exists(drive_zip):
        print(f"Found zip in Drive: {drive_zip}")
        zip_path = drive_zip
    else:
        # Upload zip file
        print("Upload colab_experiment.zip:")
        from google.colab import files
        uploaded = files.upload()
        zip_path = list(uploaded.keys())[0]
    
    # Extract
    print(f"Extracting {zip_path}...")
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall('/content')
    
    print("\nExtracted files:")
    for item in os.listdir('/content'):
        if not item.startswith('.'):
            print(f"  {item}")

## Cell 3: Check GPU and Environment

In [ ]:
import torch
import os

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("")
    print("WARNING: No GPU detected!")
    print("Go to: Runtime -> Change runtime type -> GPU")

# Set working directory
os.chdir('/content')
print(f"\nWorking directory: {os.getcwd()}")

## Cell 4: Run All 16 Experiments

This will run all 16 experiments with:
- Actor LRs: `[0.0001, 0.001, 0.01, 0.1]`
- Critic LRs: `[0.0001, 0.001, 0.01, 0.1]`

**Progress is auto-saved to Google Drive every 100 episodes.**

In [ ]:
import sys
sys.path.insert(0, '/content')
sys.path.insert(0, '/content/stopping_experiment_run')

from run_stopping_experiment_colab import run_all_experiments

# Run all experiments
run_all_experiments()

## Cell 5: Check Experiment Status

In [ ]:
import json
import os

results_dir = '/content/results/stopping_experiment'
status_file = os.path.join(results_dir, 'experiment_status.json')

if os.path.exists(status_file):
    with open(status_file) as f:
        status = json.load(f)
    print("Experiment Status:")
    print(f"  Completed: {len(status['completed'])}/16")
    if status['in_progress']:
        print(f"  In progress: {status['in_progress']}")
    print("\nCompleted experiments:")
    for exp in sorted(status['completed']):
        print(f"  - {exp}")
else:
    print("No status file found yet.")

# Also check Drive backup
drive_dir = '/content/drive/MyDrive/gradient_asymmetry_results'
if os.path.exists(drive_dir):
    print(f"\nDrive backup exists: {drive_dir}")
    contents = os.listdir(drive_dir)
    print(f"  Contents: {contents[:5]}..." if len(contents) > 5 else f"  Contents: {contents}")

## Cell 6: View Results Summary

In [ ]:
import json
import os

results_dir = '/content/results/stopping_experiment'

if os.path.exists(results_dir):
    print("Experiment Results Summary:")
    print("="*70)
    print(f"{'Actor LR':<12} {'Critic LR':<12} {'Stop Ep.':<12} {'Final Reward':<15}")
    print("-"*70)
    
    for exp_dir in sorted(os.listdir(results_dir)):
        result_file = os.path.join(results_dir, exp_dir, 'results.json')
        if os.path.exists(result_file):
            with open(result_file) as f:
                data = json.load(f)
            print(f"{data['actor_lr']:<12} {data['critic_lr']:<12} {data['stopping_episode']:<12} {data['final_reward']:<15.4f}")
    print("="*70)
else:
    print("No results directory found yet.")

## Cell 7: Download Results (Optional)

Results are auto-backed up to Drive, but you can also download a zip.

In [ ]:
import shutil
import os

results_dir = '/content/results'
output_zip = '/content/experiment_results.zip'

if os.path.exists(results_dir):
    shutil.make_archive('/content/experiment_results', 'zip', results_dir)
    print(f"Created: {output_zip}")
    print(f"Size: {os.path.getsize(output_zip) / 1024:.1f} KB")
    
    # Download
    from google.colab import files
    files.download(output_zip)
else:
    print("No results directory found.")